In [ ]:
# !pip install GML

In [ ]:
!pip install autofeat

In [ ]:
# !pip install sweetviz

In [ ]:
# !pip install heamy

In [ ]:
# !pip install optuna

In [ ]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.neural_network import MLPRegressor
from sklearn.cluster import KMeans
from sklearn.ensemble import ExtraTreesRegressor, VotingRegressor, HistGradientBoostingRegressor, BaggingRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression, LassoLars, Ridge, ElasticNet, Lasso
from sklearn.inspection import permutation_importance

from mlxtend.regressor import StackingCVRegressor

# import sweetviz as sv

# import smogn

# from heamy.dataset import Dataset
# from heamy.estimator import Regressor, Classifier
# from heamy.pipeline import ModelsPipeline

import matplotlib.pyplot as plt

import seaborn as sns

from scipy import stats

from xgboost import XGBRegressor

from catboost import CatBoostRegressor

from lightgbm import LGBMRegressor

import tqdm

from category_encoders import TargetEncoder

from scipy.stats import zscore

# import optuna

from autofeat import AutoFeatRegressor

# from pystacknet.pystacknet import StackNetRegressor

# from GML.Ghalat_Machine_Learning import Ghalat_Machine_Learning

In [ ]:
def accuracy(y_true,y_pred):
  return 100 * max(0,1-np.sqrt(mean_squared_error(y_true,y_pred)))

In [ ]:
path = '/kaggle/input/he-mlcomp/'

In [ ]:
train = pd.read_csv(path+'Train.csv')
test = pd.read_csv(path+'Test.csv')

In [ ]:
"""
test = train[5000:].copy()
test_labels = test['Attrition_rate']
test.drop(['Attrition_rate'],axis=1,inplace=True)
train = train[:5000].copy()
"""
pass

In [ ]:
test_eid = test['Employee_ID'].copy()

In [ ]:
all_data = pd.concat([train,test],axis=0)

In [ ]:
all_data['Employee_ID'] = all_data['Employee_ID'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
cols = all_data.columns[all_data.dtypes == 'object']
for col in cols:
    all_data[col].interpolate(method='linear',inplace=True)

In [ ]:
train = all_data[~pd.isna(all_data['Attrition_rate'])].copy()
test = all_data[pd.isna(all_data['Attrition_rate'])].copy()

In [ ]:
cols = train.columns[train.dtypes == 'object']
encoded = pd.DataFrame([])

X = train[cols]
y = train['Attrition_rate'].copy().values

for tr_in, val_in in KFold(shuffle = True).split(X, y):
    enc = TargetEncoder()
    enc.fit(X.iloc[tr_in], y[tr_in])
    encoded = encoded.append(enc.transform(X.iloc[val_in]))

enc = TargetEncoder()
train[cols] = enc.fit(X[cols],y)
test[cols] = enc.transform(test[cols])
train[cols] = encoded.sort_index()

"""
for col in cols:
  dummies = pd.get_dummies(train[col])
  train.drop([col],axis=1,inplace=True)
  train = pd.concat([train,dummies],axis=1)
  dummies = pd.get_dummies(test[col])
  test.drop([col],axis=1,inplace=True)
  test = pd.concat([test,dummies],axis=1)
"""
"""
for col in test.columns:
    if train[col].isnull().any():
        model = LGBMRegressor(**grid_lgbm)
        t1 = train[~pd.isna(train[col])].copy()
        t1.drop(['Attrition_rate'],axis=1,inplace=True)
        t2 = train[pd.isna(train[col])].copy()
        t2.drop(['Attrition_rate','Age'],axis=1,inplace=True)
        t3 = test[pd.isna(test[col])].copy()
        t3.drop(['Age'],axis=1,inplace=True)
        model.fit(t1.drop([col],axis=1),t1[col])
        train.loc[t2.index,col] = model.predict(t2)
        test.loc[t3.index,col] = model.predict(t3)
"""
"""
for cols in test.columns:
    train[cols].fillna(train[cols].mean(),inplace=True)
    test[cols].fillna(test[cols].mean(),inplace=True)
"""
pass

In [ ]:
test.drop(['Attrition_rate'],axis=1,inplace=True)

In [ ]:
cols = test.columns
for col in cols:
    test[col].interpolate(method='linear',inplace=True)
    train[col].interpolate(method='linear',inplace=True)

In [ ]:
X = train.drop(['Attrition_rate'],axis=1)
y = train['Attrition_rate'].copy().values

In [ ]:
af = AutoFeatRegressor(feateng_steps=2, verbose=1)

In [ ]:
X = af.fit_transform(X, y)
test = af.transform(test)

In [ ]:
nb = LGBMRegressor()
nb.fit(X, y)
"""
fi = pd.DataFrame()
fi['A'] = X.columns.values
fi['B'] = lgbm.feature_importances_
"""
pass

In [ ]:
fi = pd.DataFrame()
fi['Name'] = X.columns 
fi['importance'] = nb.feature_importances_
fi.sort_values('importance',ascending=False,inplace=True)
fi

In [ ]:
thresh = 10

In [ ]:
cols_to_d = fi[fi['importance'] < thresh].loc[:,'Name']

In [ ]:
X_BR = X.drop(cols_to_d,axis=1)
test_BR = test.drop(cols_to_d,axis=1)

In [ ]:
thresh = 100
cols_to_d = fi[fi['importance'] < thresh].loc[:,'Name']

In [ ]:
X_LR = X.drop(cols_to_d,axis=1)
test_LR = test.drop(cols_to_d,axis=1)

In [ ]:
thresh = 10
cols_to_d = fi[fi['importance'] < thresh].loc[:,'Name']

In [ ]:
X_L = X.drop(cols_to_d,axis=1)
test_L = test.drop(cols_to_d,axis=1)

In [ ]:
thresh = 10
cols_to_d = fi[fi['importance'] < thresh].loc[:,'Name']

In [ ]:
X_XGB = X.drop(cols_to_d,axis=1)
test_XGB = test.drop(cols_to_d,axis=1)

In [ ]:
thresh = 50
cols_to_d = fi[fi['importance'] < thresh].loc[:,'Name']

In [ ]:
X_lars = X.drop(cols_to_d,axis=1)
test_lars = test.drop(cols_to_d,axis=1)

In [ ]:
thresh = 50
cols_to_d = fi[fi['importance'] < thresh].loc[:,'Name']

In [ ]:
X_lgbm = X.drop(cols_to_d,axis=1)
test_lgbm = test.drop(cols_to_d,axis=1)

In [ ]:
ss = pd.DataFrame()
ss['Employee_ID'] = test_eid
ss['Attrition_rate'] = np.nan

In [ ]:
p1, p2, p3, p4 = 0.02, 0.01, 0.1, 0.91

In [ ]:
def checkPerformance(model, X, y):
    scores = []

    for tr_in, val_in in KFold(shuffle=True).split(X, y):
        model.fit(X.iloc[tr_in], y[tr_in])
        preds = model.predict(X.iloc[val_in])
        scores.append(accuracy(y[val_in],preds))
    return np.mean(scores)

In [ ]:
scores = []

for tr_in, val_in in KFold(shuffle=True).split(X_L, y):
    br = BayesianRidge()
    lr = Ridge()
    lasso = Lasso()
    lars = LassoLars()
    
    br.fit(X_BR.iloc[tr_in], y[tr_in])
    lr.fit(X_LR.iloc[tr_in], y[tr_in])
    lasso.fit(X_L.iloc[tr_in], y[tr_in])
    lars.fit(X_lars.iloc[tr_in], y[tr_in])
    
    preds = (br.predict(X_BR.iloc[val_in]) * p1 + lr.predict(X_LR.iloc[val_in]) * p2 + 
                        lasso.predict(X_L.iloc[val_in]) * p3 + lars.predict(X_lars.iloc[val_in]) * p4)
    scores.append(accuracy(y[val_in],preds))

In [ ]:
scores

In [ ]:
np.mean(scores)

In [ ]:
# sr = StackingCVRegressor([BayesianRidge(), Ridge(), Lasso(), LassoLars()], BayesianRidge())

In [ ]:
br = BayesianRidge()
lr = Ridge()
lasso = Lasso()
lars = LassoLars()

br.fit(X_BR, y)
lr.fit(X_LR, y)
lasso.fit(X_L, y)
lars.fit(X_lars, y)

pass

In [ ]:
test_L.fillna(0,inplace=True)
test_LR.fillna(0,inplace=True)
test_BR.fillna(0,inplace=True)
test_lars.fillna(0,inplace=True)
test_XGB.fillna(0,inplace=True)
test_lgbm.fillna(0,inplace=True)

In [ ]:
preds = (br.predict(test_BR) * p1 + lr.predict(test_LR) * p2 + lasso.predict(test_L) * p3 +
        lars.predict(test_lars) * p4 )

In [ ]:
ss['Attrition_rate'] = preds

In [ ]:
high = pd.read_csv('/kaggle/input/81499-he/81499.csv')['Attrition_rate']
high0 = pd.read_csv('/kaggle/input/31440-he/31440.csv')['Attrition_rate']
high1 = pd.read_csv('/kaggle/input/31412he/31412.csv')['Attrition_rate']
pred = pd.read_csv('/kaggle/input/8134704-he/8134704.csv')['Attrition_rate']
low = pd.read_csv('/kaggle/input/81290-he/81290.csv')['Attrition_rate']
low1 = pd.read_csv('/kaggle/input/81301-he/81301.csv')['Attrition_rate']

pred1 = br.predict(test_BR) 
pred2 = lr.predict(test_LR) 
pred3 = lasso.predict(test_L) 
pred4 = lars.predict(test_lars)

In [ ]:
p = 0.4 * low + 0.6 * preds 

In [ ]:
ss['Attrition_rate'] = (high + p) - (preds) 

In [ ]:
ss.to_csv('submission.csv',index=False)

In [ ]:
ss

In [ ]:
ss.describe()